In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
from sklearn.utils import shuffle

In [5]:
#since I am using colab.research.google.com have to do the follwoing to import my data set
from google.colab import files

!rm Absenteeism_preprocessed.csv
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

rm: cannot remove 'Absenteeism-data.csv': No such file or directory


Saving Absenteeism_preprocessed.csv to Absenteeism_preprocessed.csv
User uploaded file "Absenteeism_preprocessed.csv" with length 34059 bytes


##PREPROCESS

In [338]:
data = pd.read_csv('Absenteeism_preprocessed.csv')
#create targets
#consider 2 classes ; moderatley absent and excessively absent
#take mean of absenteeism hours. below median is moderatley absent else excessively absent

median_absent_time = data['Absenteeism Time in Hours'].median()

targets = np.where(data['Absenteeism Time in Hours'] > median_absent_time, 1, 0) #condition, true value, false value
#merge targets to data

data["Excessive Absenteeism"] = targets
targets = data.iloc[:,-1] #so that target is now a df too
data.head()

,Unnamed: 0,Reason 1,Reason 2,Reason 3,Reason 4,Months,Day of Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


In [339]:
data = data.drop(["Unnamed: 0","Excessive Absenteeism","Absenteeism Time in Hours"],axis=1)
data.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Months,Day of Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1


In [0]:
#balance data set as there are more 0s in targets than 1s
#so count 1s so that equal 0s can be used

target_one_count = int(np.sum(targets)) #as we have 0s and 1s in target so sum of all targets will give us count of 1s
#now traverse over data set and select indices to keep and the ones to delete.Start deleting when count for 0s becomes equal to count of 1s
target_zero_count = 0
indices_to_delete_for_balancing = []

no_of_rows_in_data_set = data.shape[0] #gets us the number of rows in data set

for i in range(no_of_rows_in_data_set):
  if(targets[i] == 0):
    target_zero_count += 1
    if(target_zero_count > target_one_count):
      indices_to_delete_for_balancing.append(i)
      
unscaled_but_balanced_data = data.drop(indices_to_delete_for_balancing, axis=0) #axis 0 as we want to delete rows
balanced_targets = targets.drop(indices_to_delete_for_balancing,axis=0)

In [341]:
#now that we have balanced data.
#we shall then scale/standardize the data. but not dummy vars
cols_to_scale =['Months','Day of Week','Transportation Expense','Distance to Work','Age','Daily Work Load Average','Body Mass Index','Children', 'Pets']

scaled_and_balanced_data = unscaled_but_balanced_data.copy()
scaled_and_balanced_data[cols_to_scale] = preprocessing.scale(unscaled_but_balanced_data[cols_to_scale]) #stadardizes all inputs. each column is treated as an array and then standardized
scaled_and_balanced_data = scaled_and_balanced_data.reset_index(drop=True)
balanced_targets = balanced_targets.reset_index(drop=True)
scaled_and_balanced_data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  after removing the cwd from sys.path.


,Reason 1,Reason 2,Reason 3,Reason 4,Months,Day of Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.102886,-0.676272,0.958760,0.418200,-0.530537,-0.890337,0.774218,0,0.827424,0.246039
1,0,0,0,0,0.102886,-0.676272,-1.591870,-1.138145,2.190129,-0.890337,1.012381,0,-0.058308,-0.597900
2,0,0,0,1,0.102886,-0.005259,-0.681996,1.433208,0.269658,-0.890337,1.012381,0,-0.944040,-0.597900
3,1,0,0,0,0.102886,0.665755,0.809600,-1.679483,0.429698,-0.890337,-0.654763,0,0.827424,-0.597900
4,0,0,0,1,0.102886,0.665755,0.958760,0.418200,-0.530537,-0.890337,0.774218,0,0.827424,0.246039


In [342]:
#merging targets to scaled_and_balanced_data so that shuffling keeps relationship between rows and target
scaled_and_balanced_data['Targets'] = balanced_targets
shuffled_scaled_and_balanced_data  = shuffle(scaled_and_balanced_data)
shuffled_balanced_targets = shuffled_scaled_and_balanced_data['Targets']
shuffled_balanced_targets.unique()
shuffled_scaled_and_balanced_data = shuffled_scaled_and_balanced_data.drop(['Targets'],axis=1)
shuffled_scaled_and_balanced_data.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Months,Day of Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
556,0,0,0,1,1.517576,-0.005259,0.988591,0.079864,0.589737,0.133662,-0.416599,0,-0.058308,0.246039
542,1,0,0,0,1.234638,0.665755,-1.591870,-1.341147,0.109619,-0.166831,0.297891,0,-0.944040,-0.597900
318,0,0,0,1,1.234638,-1.347286,0.347205,-0.326139,1.710011,0.220638,1.250545,0,0.827424,0.246039
126,0,0,0,1,-1.594741,1.336769,-1.591870,-1.341147,0.109619,0.834163,0.297891,0,-0.944040,-0.597900
152,0,0,0,1,-1.311803,-0.676272,1.570314,-0.935144,-1.330733,0.684092,-0.416599,1,-0.944040,-0.597900


In [343]:
shuffled_balanced_targets.head()

556    0
542    0
318    0
126    0
152    0
Name: Targets, dtype: int64

In [344]:
#split into training, validation and test
sample_count_of_preprocess_data = shuffled_scaled_and_balanced_data.shape[0]

training_sample_count = int(0.8*sample_count_of_preprocess_data)
validation_sample_count = int(0.1*sample_count_of_preprocess_data)
test_sample_count = sample_count_of_preprocess_data - training_sample_count - validation_sample_count

#time to divide data into these training, validation and test
training_data = shuffled_scaled_and_balanced_data[:training_sample_count] #start to training_sample_count
validation_data = shuffled_scaled_and_balanced_data[training_sample_count : training_sample_count + validation_sample_count]
test_data = shuffled_scaled_and_balanced_data[training_sample_count + validation_sample_count : ] #everything that is left

training_targets = shuffled_balanced_targets[:training_sample_count]
validation_targets = shuffled_balanced_targets[training_sample_count : training_sample_count + validation_sample_count]
test_targets = shuffled_balanced_targets[training_sample_count + validation_sample_count :]

#since we have balanced the data before spliting, it is a good idea to check here if the splitted data are balanced too. should be close to 50% for each seta s we only have 2 categories
print("% of 1s in training data: " + str(np.sum(training_targets)/training_sample_count))
print("% of 1s in validation data: " + str(np.sum(validation_targets)/validation_sample_count))
print("% of 1s in test data: " + str(np.sum(test_targets)/test_sample_count))

#also that the total count adds up
print("total: " + str(training_sample_count + validation_sample_count + test_sample_count))

% of 1s in training data: 0.492156862745098
% of 1s in validation data: 0.6190476190476191
% of 1s in test data: 0.4461538461538462
total: 638


In [0]:
#save 3 data sets in npz files
np.savez('Absenteeism_Training_Data.npz',inputs=training_data, targets = training_targets)
np.savez('Absenteeism_Validation_Data.npz',inputs=validation_data, targets = validation_targets)
np.savez('Absenteeism_Test_Data.npz',inputs=test_data, targets = test_targets)
#data now is preprocessed. So pre processing complete

##Model

In [0]:
# Create a class that will do the batching for the algorithm
# This code is extremely reusable. You should just change Audiobooks_data everywhere in the code
class Data_Reader():
    # Dataset is a mandatory arugment, while the batch_size is optional
    # If you don't input batch_size, it will automatically take the value: None
    def __init__(self, dataset, batch_size = None):
    
        # The dataset that loads is one of "train", "validation", "test".
        # e.g. if I call this class with x('train',5), it will load 'Audiobooks_data_train.npz' with a batch size of 5.
        npz = np.load('Absenteeism_{0}_Data.npz'.format(dataset))
        
        # Two variables that take the values of the inputs and the targets. Inputs are floats, targets are integers
        self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
        
        # Counts the batch number, given the size you feed it later
        # If the batch size is None, we are either validating or testing, so we want to take the data in a single batch
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
    
    # A method which loads the next batch
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
            
        # You slice the dataset in batches and then the "next" function loads them one after the other
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1
        
        # One-hot encode the targets. In this example it's a bit superfluous since we have a 0/1 column 
        # as a target already but we're giving you the code regardless, as it will be useful for any 
        # classification task with more than one target column
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        
        # The function will return the inputs batch and the one-hot encoded targets
        return inputs_batch, targets_one_hot
    
        
    # A method needed for iterating over the batches, as we will put them in a loop
    # This tells Python that the class we're defining is iterable, i.e. that we can use it like:
    # for input, output in data: 
        # do things
    # An iterator in Python is a class with a method __next__ that defines exactly how to iterate through its objects
    def __iter__(self):
        return self

In [0]:
#model

input_size = 14
output_size = 2
hidden_layer_size = 50

tf.reset_default_graph()

inputs_placeholder = tf.placeholder(tf.float32,[None,input_size])
targets_placeholder = tf.placeholder(tf.float32,[None,output_size])

#hidden layer 1
weights_1 = tf.get_variable("weigths_1",[input_size,hidden_layer_size]) #if initialized this way, the default initializer is xavier
biases_1 = tf.get_variable("biases_1",[hidden_layer_size])
output_1 = tf.nn.tanh(tf.matmul(inputs_placeholder,weights_1) + biases_1)


#output layer
weights_2 = tf.get_variable("weigths_2",[hidden_layer_size,output_size]) #if initialized this way, the default initializer is xavier
biases_2 = tf.get_variable("biases_2",[output_size])
outputs = tf.matmul(output_1,weights_2) + biases_2

In [348]:
outputs

<tf.Tensor 'add_1:0' shape=(?, 2) dtype=float32>

In [0]:
#calculate loss/delta
loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs,labels=targets_placeholder)

mean_loss = tf.reduce_mean(loss)

In [0]:
#optimize
learning_rate = 0.001
optimize = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(mean_loss)

In [0]:
#test accuracy
out_equals_target = tf.equal(tf.argmax(outputs,1),tf.argmax(targets_placeholder,1))
#out_equals_target is a vector containing 1 if accurately predcted else 0. the accuracy is the mean of this vector
accuracy = tf.reduce_mean(tf.cast(out_equals_target,tf.float32))

In [352]:
#session
session = tf.InteractiveSession()
#initializer
initializer=tf.global_variables_initializer()
session.run(initializer)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
#batching
batch_size = 50
#batch number is handled in Data_Reader class
max_epoch = 100
prev_validation_loss = 9999999999.

#get data
training_df = Data_Reader('Training',batch_size)
validation_df = Data_Reader('Validation',batch_size)

In [354]:

#train model
for epoch in range(max_epoch):
  curr_epoch_loss = 0
  #train
  for input_batch, target_batch in training_df:
    _, batch_loss = session.run([optimize,mean_loss], feed_dict = {inputs_placeholder: input_batch, targets_placeholder:target_batch})
    curr_epoch_loss += batch_loss
  
  curr_epoch_loss /= training_df.batch_count
  
  #validate
  validation_loss = 0.
  validation_accuracy = 0.
  for input_batch, target_batch in validation_df: #this will always have a single iteration but easy way to get input and target batches
    validation_loss, validation_accuracy = session.run([mean_loss,accuracy], feed_dict = {inputs_placeholder: input_batch, targets_placeholder:target_batch})
  
  
  print('Epoch '+str(epoch+1)
         + '. Training loss: '+'{0:.3f}'.format(curr_epoch_loss)
         +'. Validation loss: '+'{0:.3f}'.format(validation_loss)
         +'. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.)+'%')
  #early stop
  if validation_loss > prev_validation_loss:
    break
    
  prev_validation_loss = validation_loss
  
print('End of Training')

Epoch 1. Training loss: 0.754. Validation loss: 0.613. Validation accuracy: 66.00%
Epoch 2. Training loss: 0.689. Validation loss: 0.572. Validation accuracy: 76.00%
Epoch 3. Training loss: 0.647. Validation loss: 0.553. Validation accuracy: 78.00%
Epoch 4. Training loss: 0.618. Validation loss: 0.545. Validation accuracy: 74.00%
Epoch 5. Training loss: 0.599. Validation loss: 0.542. Validation accuracy: 74.00%
Epoch 6. Training loss: 0.584. Validation loss: 0.540. Validation accuracy: 72.00%
Epoch 7. Training loss: 0.573. Validation loss: 0.538. Validation accuracy: 74.00%
Epoch 8. Training loss: 0.564. Validation loss: 0.536. Validation accuracy: 72.00%
Epoch 9. Training loss: 0.557. Validation loss: 0.535. Validation accuracy: 72.00%
Epoch 10. Training loss: 0.552. Validation loss: 0.534. Validation accuracy: 72.00%
Epoch 11. Training loss: 0.547. Validation loss: 0.534. Validation accuracy: 72.00%
Epoch 12. Training loss: 0.544. Validation loss: 0.533. Validation accuracy: 72.00%
E

In [355]:
#test the model
test_data = Data_Reader('Test',batch_size)
for input_batch, target_batch in test_data: 
  test_accuracy = session.run([accuracy], feed_dict = {inputs_placeholder: input_batch, targets_placeholder:target_batch})
    
print('{0:3f}'.format(test_accuracy[0]*100) + '%')

72.000003%
